### DESCRIPTION:
    Use Azure OpenAI GPT3.5 with few shot learning to imitate someone’s writing style
### REQUIREMENTS:
    Create an .env file with your OpenAI API key and save it in the root directory of this project.


#### **Using LangChain for Few-Shot learning**

In [12]:
from langchain import FewShotPromptTemplate
from langchain import PromptTemplate
from langchain.llms import AzureOpenAI
from langchain import LLMChain
import openai
import os
from dotenv import load_dotenv
from IPython.display import display, HTML, JSON
import utils

In [13]:
def init_llm(model=utils.OPENAI_MODEL_NAME,
             deployment_name=utils.OPENAI_DEPLOYMENT_NAME, 
             temperature=0,
             max_tokens=400,
             stop="<|im_end|>", 
             ):
    
    llm = AzureOpenAI(deployment_name=deployment_name,  
                  model=model,
                  temperature=temperature,) 
    return llm

In [14]:
llm=init_llm()

In [18]:
# create few shot examples
examples = [
    {
        "query": "Please write an email to a client with a business proposal.",
        "answer": """Dear Rachel Rhohes,
        My name is Robert Smith, and I'm contacting you from Acme Corp. Thanks for taking the time to chat with me earlier this week, here's my business proposal.
        At Acme Corp, we partner with leading businesses such as yours, helping them improve the services they offer and save money. By signing up to work with us, we can streamline the service you receive, making them quicker and more efficient.
        Working together, we estimate we can save you up to 50% on your annual fees, and all for just $10000.
        Please review my proposal and come back to me with any questions. The next step is to formalize our agreement and sign a contract. We're ready when you are.
        Yours Sincerely,
        Robert Smith"""
    }, 
    {
        "query": "Please write an email to a client with a business proposal.",
        "answer": """Dear Jackie MCdonald,
        My name is Robert Smith, and I'm contacting you from Acme Corp. Thanks for allowing us to submit a proposal for this exciting opportunity.
        We understand you seek to appoint a new supplier and cut costs on your current contract, and we can help. In our proposal, you will find details of how we can streamline your service, saving you money while delivering the same exceptional service levels. Within the proposal, you will find a breakdown of all contract fees. We've worked hard to unlock efficiencies and deliver everything you've asked for within the cost envelope.
        Please review my proposal and come back to me if you have any questions.

        Yours Sincerely,
        Robert Smith"""
    },
    {
        "query": "Please write an email to a client with a business proposal.",
        "answer": """
        Hi John,
        Thanks for discussing your business requirements with me. I know that cutting costs is your biggest concern, and I'm pleased to say that we can help you.
        We've calculated that we can save you up to 50% on the price you pay your current supplier.
        You may be worried about how we can be so cheap, but you should be. We have a history of working with businesses like yours and are happy to share references and case studies.
        Please review our proposal and contact me if you have any questions or require additional information.
        I look forward to hearing from you about the next steps and potentially working together in the future.

        Thank you,
        Robert Smith"""
    },
    
]

# create an example template
example_template = """
User: {query}
Assistant: {answer}
"""

# create a prompt example from the template above
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix_template = """You are an {profession} and you are writing an email about {subject}. Here are some examples: 
"""
# and the suffix our user input and output indicator
suffix_template = """
User: {query}
Assistant: """

prefix_prompt = PromptTemplate(input_variables=["profession", "subject"], template=prefix_template)

# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix = prefix_prompt.format(profession="Assistant", subject="a business proposal"),
    suffix=suffix_template,
    input_variables=["query"],
    example_separator="\n\n"
)

In [17]:
chain = LLMChain(llm=llm, prompt=few_shot_prompt_template)
ans = chain.run(few_shot_prompt_template.format(query=query))
display (HTML(ans))